# Preselection 

## Import libraries and choose whether to run locally or distributed

In [1]:
%%capture

import ROOT
import os
from dask.distributed import Client
import json
from samples import *

distributed = False
MT = False
redirector = "xrootd-cms.infn.it" 
maxNfilespersample = 2

Error in <TExMap::Add>: key 140654153440432 is not unique
Error in <TExMap::Add>: key 140654153965264 is not unique


In [2]:
#%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/workspace/persistent-storage/JMEfinal/
#!cp /opt/workspace/persistent-storage/JMEfinal/libCMSJMECalculatorsDict.so .
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepFlavour_94XSF_V3_B_F.csv", "DeepFlavour_94XSF_V3_B_F.csv");') 

In [3]:
'''
%%capture
from CMSJMECalculators import loadJMESystematicsCalculators
loadJMESystematicsCalculators()

import ROOT as gbl
gbl.gROOT.ProcessLine("JetVariationsCalculator myJetVarCalc{};")
calc = getattr(gbl, "myJetVarCalc")
calc = gbl.JetVariationsCalculator()

# redo JEC, push_back corrector parameters for different levels
jecParams = getattr(gbl, "std::vector<JetCorrectorParameters>")()
jecParams.push_back(gbl.JetCorrectorParameters(textfilepath))
calc.setJEC(jecParams)

# calculate JES uncertainties (repeat for all sources)
jcp_unc = gbl.JetCorrectorParameters(textfilepath_UncertaintySources)
calc.addJESUncertainty("Total", jcp_unc)

# Smear jets, with JER uncertainty
calc.setSmearing(textfilepath_PtResolution, textfilepath_SF, splitJER, True, 0.2, 3.)  # use hybrid recipe, matching parameters

df.Define("ak4JetVars", "myJetVarcalc.produce(Jet_pt, Jet_eta, Jet_phi, ...)")
'''

'\n%%capture\nfrom CMSJMECalculators import loadJMESystematicsCalculators\nloadJMESystematicsCalculators()\n\nimport ROOT as gbl\ngbl.gROOT.ProcessLine("JetVariationsCalculator myJetVarCalc{};")\ncalc = getattr(gbl, "myJetVarCalc")\ncalc = gbl.JetVariationsCalculator()\n\n# redo JEC, push_back corrector parameters for different levels\njecParams = getattr(gbl, "std::vector<JetCorrectorParameters>")()\njecParams.push_back(gbl.JetCorrectorParameters(textfilepath))\ncalc.setJEC(jecParams)\n\n# calculate JES uncertainties (repeat for all sources)\njcp_unc = gbl.JetCorrectorParameters(textfilepath_UncertaintySources)\ncalc.addJESUncertainty("Total", jcp_unc)\n\n# Smear jets, with JER uncertainty\ncalc.setSmearing(textfilepath_PtResolution, textfilepath_SF, splitJER, True, 0.2, 3.)  # use hybrid recipe, matching parameters\n\ndf.Define("ak4JetVars", "myJetVarcalc.produce(Jet_pt, Jet_eta, Jet_phi, ...)")\n'

In [4]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

## Declare utils defined in a preselection.h file to the ROOT interpreter. If running distributed, connect to cluster and declare the file content in an initialization function

In [5]:
def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))

In [6]:
ROOT.gInterpreter.AddIncludePath("/cvmfs/cms.dodas.infn.it/boost/include")
#ROOT.gInterpreter.ProcessLine('gSystem->AddLinkedLibs("-L/my/path -l*anylib*");')

In [7]:
#ROOT.gInterpreter.Declare("#include <boost/math/special_functions/erf.hpp>")
#ROOT.gInterpreter.ProcessLine("erf(40);")

In [8]:
ROOT.gInterpreter.Declare('''
#include <curl/curl.h>

void download_file(const char* url, const char* file_name)
{
  CURL* easyhandle = curl_easy_init();
  curl_easy_setopt( easyhandle, CURLOPT_URL, url ) ;
  FILE* file = fopen( file_name, "w");
  curl_easy_setopt( easyhandle, CURLOPT_WRITEDATA, file) ;
  curl_easy_perform( easyhandle );
  curl_easy_cleanup( easyhandle );
  fclose(file);
}
''')
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v3/RoccoR2016.txt", "RoccoR2016.txt");') 
ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v3/RoccoR2017.txt", "RoccoR2017.txt");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v3/RoccoR2018.txt", "RoccoR2018.txt");') 
ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v3/RoccoR.cc", "RoccoR.cc");')
ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v3/RoccoR.h", "RoccoR.h");')
ROOT.gInterpreter.Declare('#include "RoccoR.cc"')
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_2016LegacySF_V1.csv", "DeepJet_2016LegacySF_V1.csv");')
ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepFlavour_94XSF_V3_B_F.csv", "DeepFlavour_94XSF_V3_B_F.csv");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_102XSF_V1.csv", "DeepJet_102XSF_V1.csv");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_106XUL16preVFPSF_v1_new.csv", "DeepJet_106XUL16preVFPSF_v1_new.csv");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_106XUL16postVFPSF_v2_new.csv", "DeepJet_106XUL16postVFPSF_v2_new.csv");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_106XUL17_v3_new.csv", "DeepJet_106XUL17_v3_new.csv");') 
#ROOT.gInterpreter.ProcessLine('download_file("https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/data/btagSF/DeepJet_106XUL18_v2_new.csv", "DeepJet_106XUL18_v2_new.csv");') 

0

In [9]:
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    sched_port = 38651
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    text_file = open("preselection.h", "r")
    data = text_file.read()
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    ROOT.gInterpreter.Declare('#include "preselection.h"')

Loading histogram NUM_TightID_DEN_genTracks_eta_pt from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Mu_RunBCDEFGH_SF_ID_2016_syst.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/EGM2D_RECO_SF_2016.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/2016LegacyReReco_ElectronMVA90noiso_Fall17V2.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools

In [10]:
#ROOT.gInterpreter.ProcessLine('reader_0_Legacy2016.load(calibration_Legacy2016, 0, "comb");')
ROOT.gInterpreter.ProcessLine(
'''
reader_0_2017.load(calibration_2017, BTagEntry::FLAV_B, "comb"); //0 is flavor_btv
reader_0_2017.load(calibration_2017, BTagEntry::FLAV_C, "comb");
reader_0_2017.load(calibration_2017, BTagEntry::FLAV_UDSG, "incl");
reader_1_2017.load(calibration_2017, BTagEntry::FLAV_B, "comb"); //0 is flavor_btv
reader_1_2017.load(calibration_2017, BTagEntry::FLAV_C, "comb");
reader_1_2017.load(calibration_2017, BTagEntry::FLAV_UDSG, "incl");
reader_2_2017.load(calibration_2017, BTagEntry::FLAV_B, "comb"); //0 is flavor_btv
reader_2_2017.load(calibration_2017, BTagEntry::FLAV_C, "comb");
reader_2_2017.load(calibration_2017, BTagEntry::FLAV_UDSG, "incl");
''')

'''
reader_0_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv
reader_0_Legacy2016.load(calibration_Legacy2016, 1, "comb");
reader_0_Legacy2016.load(calibration_Legacy2016, 2, "incl");
reader_1_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv
reader_1_Legacy2016.load(calibration_Legacy2016, 1, "comb");
reader_1_Legacy2016.load(calibration_Legacy2016, 2, "incl");
reader_2_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv
reader_2_Legacy2016.load(calibration_Legacy2016, 1, "comb");
reader_2_Legacy2016.load(calibration_Legacy2016, 2, "incl");

reader_0_2017.load(calibration_2017, 0, "comb"); //0 is flavor_btv
reader_0_2017.load(calibration_2017, 1, "comb");
reader_0_2017.load(calibration_2017, 2, "incl");
reader_1_2017.load(calibration_2017, 0, "comb"); //0 is flavor_btv
reader_1_2017.load(calibration_2017, 1, "comb");
reader_1_2017.load(calibration_2017, 2, "incl");
reader_2_2017.load(calibration_2017, 0, "comb"); //0 is flavor_btv
reader_2_2017.load(calibration_2017, 1, "comb");
reader_2_2017.load(calibration_2017, 2, "incl");

reader_0_2018.load(calibration_2018, 0, "comb"); //0 is flavor_btv
reader_0_2018.load(calibration_2018, 1, "comb");
reader_0_2018.load(calibration_2018, 2, "incl");
reader_1_2018.load(calibration_2018, 0, "comb"); //0 is flavor_btv
reader_1_2018.load(calibration_2018, 1, "comb");
reader_1_2018.load(calibration_2018, 2, "incl");
reader_2_2018.load(calibration_2018, 0, "comb"); //0 is flavor_btv
reader_2_2018.load(calibration_2018, 1, "comb");
reader_2_2018.load(calibration_2018, 2, "incl");

reader_0_UL2016APV.load(calibration_UL2016APV, 0, "comb"); //0 is flavor_btv
reader_0_UL2016APV.load(calibration_UL2016APV, 1, "comb");
reader_0_UL2016APV.load(calibration_UL2016APV, 2, "incl");
reader_1_UL2016APV.load(calibration_UL2016APV, 0, "comb"); //0 is flavor_btv
reader_1_UL2016APV.load(calibration_UL2016APV, 1, "comb");
reader_1_UL2016APV.load(calibration_UL2016APV, 2, "incl");
reader_2_UL2016APV.load(calibration_UL2016APV, 0, "comb"); //0 is flavor_btv
reader_2_UL2016APV.load(calibration_UL2016APV, 1, "comb");
reader_2_UL2016APV.load(calibration_UL2016APV, 2, "incl");

reader_0_UL2016.load(calibration_UL2016, 0, "comb"); //0 is flavor_btv
reader_0_UL2016.load(calibration_UL2016, 1, "comb");
reader_0_UL2016.load(calibration_UL2016, 2, "incl");
reader_1_UL2016.load(calibration_UL2016, 0, "comb"); //0 is flavor_btv
reader_1_UL2016.load(calibration_UL2016, 1, "comb");
reader_1_UL2016.load(calibration_UL2016, 2, "incl");
reader_2_UL2016.load(calibration_UL2016, 0, "comb"); //0 is flavor_btv
reader_2_UL2016.load(calibration_UL2016, 1, "comb");
reader_2_UL2016.load(calibration_UL2016, 2, "incl");

reader_0_UL2017.load(calibration_UL2017, 0, "comb"); //0 is flavor_btv
reader_0_UL2017.load(calibration_UL2017, 1, "comb");
reader_0_UL2017.load(calibration_UL2017, 2, "incl");
reader_1_UL2017.load(calibration_UL2017, 0, "comb"); //0 is flavor_btv
reader_1_UL2017.load(calibration_UL2017, 1, "comb");
reader_1_UL2017.load(calibration_UL2017, 2, "incl");
reader_2_UL2017.load(calibration_UL2017, 0, "comb"); //0 is flavor_btv
reader_2_UL2017.load(calibration_UL2017, 1, "comb");
reader_2_UL2017.load(calibration_UL2017, 2, "incl");

reader_0_UL2018.load(calibration_UL2018, 0, "comb"); //0 is flavor_btv
reader_0_UL2018.load(calibration_UL2018, 1, "comb");
reader_0_UL2018.load(calibration_UL2018, 2, "incl");
reader_1_UL2018.load(calibration_UL2018, 0, "comb"); //0 is flavor_btv
reader_1_UL2018.load(calibration_UL2018, 1, "comb");
reader_1_UL2018.load(calibration_UL2018, 2, "incl");
reader_2_UL2018.load(calibration_UL2018, 0, "comb"); //0 is flavor_btv
reader_2_UL2018.load(calibration_UL2018, 1, "comb");
reader_2_UL2018.load(calibration_UL2018, 2, "incl");
'''

'\nreader_0_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv\nreader_0_Legacy2016.load(calibration_Legacy2016, 1, "comb");\nreader_0_Legacy2016.load(calibration_Legacy2016, 2, "incl");\nreader_1_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv\nreader_1_Legacy2016.load(calibration_Legacy2016, 1, "comb");\nreader_1_Legacy2016.load(calibration_Legacy2016, 2, "incl");\nreader_2_Legacy2016.load(calibration_Legacy2016, 0, "comb"); //0 is flavor_btv\nreader_2_Legacy2016.load(calibration_Legacy2016, 1, "comb");\nreader_2_Legacy2016.load(calibration_Legacy2016, 2, "incl");\n\nreader_0_2017.load(calibration_2017, 0, "comb"); //0 is flavor_btv\nreader_0_2017.load(calibration_2017, 1, "comb");\nreader_0_2017.load(calibration_2017, 2, "incl");\nreader_1_2017.load(calibration_2017, 0, "comb"); //0 is flavor_btv\nreader_1_2017.load(calibration_2017, 1, "comb");\nreader_1_2017.load(calibration_2017, 2, "incl");\nreader_2_2017.load(calibration_2017, 0, "comb")

In [11]:
ROOT.gInterpreter.Declare('#include "preselection_part2.h"')

True

In [12]:
sampleDict = {'WpWpJJ_EWK_2017': 0, 'WpWpJJ_QCD_2017': 1, 'VBS_SSWW_SM_2017': 2, 'VBS_SSWW_LL_SM_2017': 3, 'VBS_SSWW_TL_SM_2017': 4, 'VBS_SSWW_TT_SM_2017': 5, 'VBS_SSWW_cHW_2017': 6, 'VBS_SSWW_cHW_BSM_2017': 7, 'VBS_SSWW_cHW_INT_2017': 8, 'VBS_SSWW_cW_2017': 9, 'VBS_SSWW_cW_BSM_2017': 10, 'VBS_SSWW_cW_INT_2017': 11, 'VBS_SSWW_cW_cHW_2017': 12, 'VBS_SSWW_DIM6_2017': 13, 'VBS_SSWW_DIM6_SM_2017': 14, 'VBS_SSWW_cHW_SM_2017': 15, 'VBS_SSWW_cW_SM_2017': 16, 'VBS_SSWW_aQGC_2017': 17, 'VBS_SSWW_FS0_25_BSM_2017': 18, 'VBS_SSWW_FS0_25_INT_2017': 19, 'VBS_SSWW_FS0_25_SM_2017': 20, 'VBS_SSWW_FS0_25_2017': 21, 'VBS_SSWW_FS0_5_BSM_2017': 22, 'VBS_SSWW_FS0_5_INT_2017': 23, 'VBS_SSWW_FS0_5_SM_2017': 24, 'VBS_SSWW_FS0_5_2017': 25, 'VBS_SSWW_FS0_0_2017': 26, 'VBS_SSWW_FS1_50_BSM_2017': 27, 'VBS_SSWW_FS1_50_INT_2017': 28, 'VBS_SSWW_FS1_50_SM_2017': 29, 'VBS_SSWW_FS1_50_2017': 30, 'VBS_SSWW_FS1_10_BSM_2017': 31, 'VBS_SSWW_FS1_10_INT_2017': 32, 'VBS_SSWW_FS1_10_SM_2017': 33, 'VBS_SSWW_FS1_10_2017': 34, 'VBS_SSWW_FS1_0_2017': 35, 'VBS_SSWW_FM0_25_BSM_2017': 36, 'VBS_SSWW_FM0_25_INT_2017': 37, 'VBS_SSWW_FM0_25_SM_2017': 38, 'VBS_SSWW_FM0_25_2017': 39, 'VBS_SSWW_FM0_5_BSM_2017': 40, 'VBS_SSWW_FM0_5_INT_2017': 41, 'VBS_SSWW_FM0_5_SM_2017': 42, 'VBS_SSWW_FM0_5_2017': 43, 'VBS_SSWW_FM0_0_2017': 44, 'VBS_SSWW_FM1_25_BSM_2017': 45, 'VBS_SSWW_FM1_25_INT_2017': 46, 'VBS_SSWW_FM1_25_SM_2017': 47, 'VBS_SSWW_FM1_25_2017': 48, 'VBS_SSWW_FM1_5_BSM_2017': 49, 'VBS_SSWW_FM1_5_INT_2017': 50, 'VBS_SSWW_FM1_5_SM_2017': 51, 'VBS_SSWW_FM1_5_2017': 52, 'VBS_SSWW_FM1_0_2017': 53, 'VBS_SSWW_FM6_25_BSM_2017': 54, 'VBS_SSWW_FM6_25_INT_2017': 55, 'VBS_SSWW_FM6_25_SM_2017': 56, 'VBS_SSWW_FM6_25_2017': 57, 'VBS_SSWW_FM6_5_BSM_2017': 58, 'VBS_SSWW_FM6_5_INT_2017': 59, 'VBS_SSWW_FM6_5_SM_2017': 60, 'VBS_SSWW_FM6_5_2017': 61, 'VBS_SSWW_FM6_0_2017': 62, 'VBS_SSWW_FM7_50_BSM_2017': 63, 'VBS_SSWW_FM7_50_INT_2017': 64, 'VBS_SSWW_FM7_50_SM_2017': 65, 'VBS_SSWW_FM7_50_2017': 66, 'VBS_SSWW_FM7_10_BSM_2017': 67, 'VBS_SSWW_FM7_10_INT_2017': 68, 'VBS_SSWW_FM7_10_SM_2017': 69, 'VBS_SSWW_FM7_10_2017': 70, 'VBS_SSWW_FM7_0_2017': 71, 'VBS_SSWW_FT0_2p5_BSM_2017': 72, 'VBS_SSWW_FT0_2p5_INT_2017': 73, 'VBS_SSWW_FT0_2p5_SM_2017': 74, 'VBS_SSWW_FT0_2p5_2017': 75, 'VBS_SSWW_FT0_0p5_BSM_2017': 76, 'VBS_SSWW_FT0_0p5_INT_2017': 77, 'VBS_SSWW_FT0_0p5_SM_2017': 78, 'VBS_SSWW_FT0_0p5_2017': 79, 'VBS_SSWW_FT0_0_2017': 80, 'VBS_SSWW_FT1_1_BSM_2017': 81, 'VBS_SSWW_FT1_1_INT_2017': 82, 'VBS_SSWW_FT1_1_SM_2017': 83, 'VBS_SSWW_FT1_1_2017': 84, 'VBS_SSWW_FT1_0p2_BSM_2017': 85, 'VBS_SSWW_FT1_0p2_INT_2017': 86, 'VBS_SSWW_FT1_0p2_SM_2017': 87, 'VBS_SSWW_FT1_0p2_2017': 88, 'VBS_SSWW_FT1_0_2017': 89, 'VBS_SSWW_FT2_2p5_BSM_2017': 90, 'VBS_SSWW_FT2_2p5_INT_2017': 91, 'VBS_SSWW_FT2_2p5_SM_2017': 92, 'VBS_SSWW_FT2_2p5_2017': 93, 'VBS_SSWW_FT2_0p5_BSM_2017': 94, 'VBS_SSWW_FT2_0p5_INT_2017': 95, 'VBS_SSWW_FT2_0p5_SM_2017': 96, 'VBS_SSWW_FT2_0p5_2017': 97, 'VBS_SSWW_FT2_0_2017': 98, 'DHiggsToWW_2017': 99, 'sm_2017': 100, 'sm_lin_quad_cW_2017': 101, 'quad_cW_2017': 102, 'sm_lin_quad_cHW_2017': 103, 'quad_cHW_2017': 104, 'QCD_2017': 105, 'QCDHT_100to200_2017': 106, 'QCDHT_200to300_2017': 107, 'QCDHT_300to500_2017': 108, 'QCDHT_500to700_2017': 109, 'QCDHT_700to1000_2017': 110, 'QCDHT_1000to1500_2017': 111, 'QCDHT_1500to2000_2017': 112, 'QCDHT_2000toInf_2017': 113, 'ZZtoLep_2017': 114, 'ZZTo2L2Nu_2017': 115, 'ZZJJTo4L_EWK_2017': 116, 'ZZJJTo4L_QCD_2017': 117, 'GluGluToContinToZZTo2e2nu_2017': 118, 'GluGluToContinToZZTo2e2mu_2017': 119, 'GluGluToContinToZZTo2e2tau_2017': 120, 'GluGluToContinToZZTo2mu2nu_2017': 121, 'GluGluToContinToZZTo2mu2tau_2017': 122, 'GluGluToContinToZZTo4e_2017': 123, 'GluGluToContinToZZTo4mu_2017': 124, 'GluGluToContinToZZTo4tau_2017': 125, 'GluGluToContinToZZTo4L_2017': 126, 'TT_2017': 127, 'TT_SemiLep2017': 128, 'TT_Had_2017': 129, 'TTTo2L2Nu_2017': 130, 'TT_beff_2017': 131, 'WJets_2017': 132, 'WJetsHT70to100_2017': 133, 'WJetsHT100to200_2017': 134, 'WJetsHT200to400_2017': 135, 'WJetsHT400to600_2017': 136, 'WJetsHT600to800_2017': 137, 'WJetsHT800to1200_2017': 138, 'WJetsHT1200to2500_2017': 139, 'WJetsHT2500toInf_2017': 140, 'WJets_Fake_2017': 141, 'WJetsHT70to100_Fake_2017': 142, 'WJetsHT100to200_Fake_2017': 143, 'WJetsHT200to400_Fake_2017': 144, 'WJetsHT400to600_Fake_2017': 145, 'WJetsHT600to800_Fake_2017': 146, 'WJetsHT800to1200_Fake_2017': 147, 'WJetsHT1200to2500_Fake_2017': 148, 'WJetsHT2500toInf_Fake_2017': 149, 'DYJetsToLL_2017': 150, 'DY1JetsToLL_2017': 151, 'DY2JetsToLL_2017': 152, 'DY3JetsToLL_2017': 153, 'DY4JetsToLL_2017': 154, 'DYJetsToLLM5to50_2017': 155, 'DYJetsToLL_Fake_2017': 156, 'DY1JetsToLL_Fake_2017': 157, 'DY2JetsToLL_Fake_2017': 158, 'DY3JetsToLL_Fake_2017': 159, 'DY4JetsToLL_Fake_2017': 160, 'VG_2017': 161, 'ZG_2017': 162, 'WG_2017': 163, 'TVX_2017': 164, 'TTGJets_2017': 165, 'TTZToQQ_2017': 166, 'TTZToLLNuNu_2017': 167, 'TTWJetsToQQ_2017': 168, 'TTWJetsToLNu_2017': 169, 'tZq_ll_4f_2017': 170, 'WrongSign_2017': 171, 'WWto2L2Nu_2017': 172, 'GluGluToWWToENuENu_2017': 173, 'GluGluToWWToENuMNu2017': 174, 'GluGluToWWToENuTNu2017': 175, 'GluGluToWWToMNuENu_2017': 176, 'GluGluToWWToMNuMNu2017': 177, 'GluGluToWWToMNuTNu2017': 178, 'GluGluToWWToTNuENu_2017': 179, 'GluGluToWWToTNuMNu2017': 180, 'GluGluToWWToTNuTNu2017': 181, 'STtW_top_2017': 182, 'GluGluHToWWTo2L2Nu_2017': 183, 'GluGluHToZZTo2L2Q_2017': 184, 'GluGluHToZZTo4L_2017': 185, 'GluGluHToTauTau_2017': 186, 'VBFHToWWTo2L2Nu_2017': 187, 'VBFHToTauTau_2017': 188, 'ttHToNonbb_2017': 189, 'VHToNonbb_2017': 190, 'OtherWS_2017': 191, 'Other_2017': 192, 'WWTo2L2Nu_DoubleScattering_2017': 193, 'WWW_4F_2017': 194, 'WWZTo3L1Nu2Q_2017': 195, 'WZZ_2017': 196, 'ZZZ_2017': 197, 'WWG_2017': 198, 'WZ_2017': 199, 'FakeMu_2017': 200, 'DataMu_2017': 201, 'DataMuB_2017': 202, 'DataMuC_2017': 203, 'DataMuD_2017': 204, 'DataMuE_2017': 205, 'DataMuF_2017': 206, 'DataMuFake_2017': 207, 'FakeEle_2017': 208, 'FakeMuPromptTau_2017': 209, 'PromptMuFakeTau_2017': 210, 'FakeMuFakeTau_2017': 211, 'DataEle_2017': 212, 'DataEleB_2017': 213, 'DataEleC_2017': 214, 'DataEleD_2017': 215, 'DataEleE_2017': 216, 'DataEleF_2017': 217, 'DataEleFake_2017': 218, 'MCFake_2017': 219, 'FakeElePromptTau_2017': 220, 'PromptEleFakeTau_2017': 221, 'FakeEleFakeTau_2017': 222, 'DataEleMu_2017': 223, 'FakeEleMu_2017': 224, 'DataTau_2017': 225, 'DataTauB_2017': 226, 'DataTauC_2017': 227, 'DataTauD_2017': 228, 'DataTauE_2017': 229, 'DataTauF_2017': 230, 'DataTauFake_2017': 231, 'DataHT_2017': 232, 'DataHTnoB_2017': 233, 'DataHTB_2017': 234, 'DataHTC_2017': 235, 'DataHTD_2017': 236, 'DataHTE_2017': 237, 'DataHTF_2017': 238, 'DataMET_2017': 239, 'DataMETB_2017': 240, 'DataMETC_2017': 241, 'DataMETD_2017': 242, 'DataMETE_2017': 243, 'DataMETF_2017': 244, 'SampleEleFake_2017': 245, 'SampleMuFake_2017': 246, 'SampleHTFake_2017': 247, 'SampleHTFakepart_2017': 248, 'WpWpJJ_EWK_2018': 249, 'WpWpJJ_QCD_2018': 250, 'VBS_SSWW_SM_2018': 251, 'VBS_SSWW_LL_SM_2018': 252, 'VBS_SSWW_TL_SM_2018': 253, 'VBS_SSWW_TT_SM_2018': 254, 'VBS_SSWW_cHW_2018': 255, 'VBS_SSWW_cHW_BSM_2018': 256, 'VBS_SSWW_cHW_INT_2018': 257, 'VBS_SSWW_cW_2018': 258, 'VBS_SSWW_cW_BSM_2018': 259, 'VBS_SSWW_cW_INT_2018': 260, 'VBS_SSWW_cW_cHW_2018': 261, 'VBS_SSWW_DIM6_2018': 262, 'VBS_SSWW_DIM6_SM_2018': 263, 'VBS_SSWW_cHW_SM_2018': 264, 'VBS_SSWW_cW_SM_2018': 265, 'VBS_SSWW_aQGC_2018': 266, 'VBS_SSWW_FS0_25_BSM_2018': 267, 'VBS_SSWW_FS0_25_INT_2018': 268, 'VBS_SSWW_FS0_25_SM_2018': 269, 'VBS_SSWW_FS0_25_2018': 270, 'VBS_SSWW_FS0_5_BSM_2018': 271, 'VBS_SSWW_FS0_5_INT_2018': 272, 'VBS_SSWW_FS0_5_SM_2018': 273, 'VBS_SSWW_FS0_5_2018': 274, 'VBS_SSWW_FS0_0_2018': 275, 'VBS_SSWW_FS1_50_BSM_2018': 276, 'VBS_SSWW_FS1_50_INT_2018': 277, 'VBS_SSWW_FS1_50_SM_2018': 278, 'VBS_SSWW_FS1_50_2018': 279, 'VBS_SSWW_FS1_10_BSM_2018': 280, 'VBS_SSWW_FS1_10_INT_2018': 281, 'VBS_SSWW_FS1_10_SM_2018': 282, 'VBS_SSWW_FS1_10_2018': 283, 'VBS_SSWW_FS1_0_2018': 284, 'VBS_SSWW_FM0_25_BSM_2018': 285, 'VBS_SSWW_FM0_25_INT_2018': 286, 'VBS_SSWW_FM0_25_SM_2018': 287, 'VBS_SSWW_FM0_25_2018': 288, 'VBS_SSWW_FM0_5_BSM_2018': 289, 'VBS_SSWW_FM0_5_INT_2018': 290, 'VBS_SSWW_FM0_5_SM_2018': 291, 'VBS_SSWW_FM0_5_2018': 292, 'VBS_SSWW_FM0_0_2018': 293, 'VBS_SSWW_FM1_25_BSM_2018': 294, 'VBS_SSWW_FM1_25_INT_2018': 295, 'VBS_SSWW_FM1_25_SM_2018': 296, 'VBS_SSWW_FM1_25_2018': 297, 'VBS_SSWW_FM1_5_BSM_2018': 298, 'VBS_SSWW_FM1_5_INT_2018': 299, 'VBS_SSWW_FM1_5_SM_2018': 300, 'VBS_SSWW_FM1_5_2018': 301, 'VBS_SSWW_FM1_0_2018': 302, 'VBS_SSWW_FM6_25_BSM_2018': 303, 'VBS_SSWW_FM6_25_INT_2018': 304, 'VBS_SSWW_FM6_25_SM_2018': 305, 'VBS_SSWW_FM6_25_2018': 306, 'VBS_SSWW_FM6_5_BSM_2018': 307, 'VBS_SSWW_FM6_5_INT_2018': 308, 'VBS_SSWW_FM6_5_SM_2018': 309, 'VBS_SSWW_FM6_5_2018': 310, 'VBS_SSWW_FM6_0_2018': 311, 'VBS_SSWW_FM7_50_BSM_2018': 312, 'VBS_SSWW_FM7_50_INT_2018': 313, 'VBS_SSWW_FM7_50_SM_2018': 314, 'VBS_SSWW_FM7_50_2018': 315, 'VBS_SSWW_FM7_10_BSM_2018': 316, 'VBS_SSWW_FM7_10_INT_2018': 317, 'VBS_SSWW_FM7_10_SM_2018': 318, 'VBS_SSWW_FM7_10_2018': 319, 'VBS_SSWW_FM7_0_2018': 320, 'VBS_SSWW_FT0_2p5_BSM_2018': 321, 'VBS_SSWW_FT0_2p5_INT_2018': 322, 'VBS_SSWW_FT0_2p5_SM_2018': 323, 'VBS_SSWW_FT0_2p5_2018': 324, 'VBS_SSWW_FT0_0p5_BSM_2018': 325, 'VBS_SSWW_FT0_0p5_INT_2018': 326, 'VBS_SSWW_FT0_0p5_SM_2018': 327, 'VBS_SSWW_FT0_0p5_2018': 328, 'VBS_SSWW_FT0_0_2018': 329, 'VBS_SSWW_FT1_1_BSM_2018': 330, 'VBS_SSWW_FT1_1_INT_2018': 331, 'VBS_SSWW_FT1_1_SM_2018': 332, 'VBS_SSWW_FT1_1_2018': 333, 'VBS_SSWW_FT1_0p2_BSM_2018': 334, 'VBS_SSWW_FT1_0p2_INT_2018': 335, 'VBS_SSWW_FT1_0p2_SM_2018': 336, 'VBS_SSWW_FT1_0p2_2018': 337, 'VBS_SSWW_FT1_0_2018': 338, 'VBS_SSWW_FT2_2p5_BSM_2018': 339, 'VBS_SSWW_FT2_2p5_INT_2018': 340, 'VBS_SSWW_FT2_2p5_SM_2018': 341, 'VBS_SSWW_FT2_2p5_2018': 342, 'VBS_SSWW_FT2_0p5_BSM_2018': 343, 'VBS_SSWW_FT2_0p5_INT_2018': 344, 'VBS_SSWW_FT2_0p5_SM_2018': 345, 'VBS_SSWW_FT2_0p5_2018': 346, 'VBS_SSWW_FT2_0_2018': 347, 'DHiggsToWW_2018': 348, 'sm_2018': 349, 'sm_lin_quad_cW_2018': 350, 'quad_cW_2018': 351, 'sm_lin_quad_cHW_2018': 352, 'quad_cHW_2018': 353, 'QCD_2018': 354, 'QCDHT_100to200_2018': 355, 'QCDHT_200to300_2018': 356, 'QCDHT_300to500_2018': 357, 'QCDHT_500to700_2018': 358, 'QCDHT_700to1000_2018': 359, 'QCDHT_1000to1500_2018': 360, 'QCDHT_1500to2000_2018': 361, 'QCDHT_2000toInf_2018': 362, 'ZZtoLep_2018': 363, 'ZZTo2L2Nu_2018': 364, 'ZZJJTo4L_EWK_2018': 365, 'ZZJJTo4L_QCD_2018': 366, 'GluGluToContinToZZTo2e2nu_2018': 367, 'GluGluToContinToZZTo2e2mu_2018': 368, 'GluGluToContinToZZTo2e2tau_2018': 369, 'GluGluToContinToZZTo2mu2nu_2018': 370, 'GluGluToContinToZZTo2mu2tau_2018': 371, 'GluGluToContinToZZTo4e_2018': 372, 'GluGluToContinToZZTo4mu_2018': 373, 'GluGluToContinToZZTo4tau_2018': 374, 'GluGluToContinToZZTo4L_2018': 375, 'TT_2018': 376, 'TT_SemiLep_2018': 377, 'TT_Had_2018': 378, 'WJets_2018': 379, 'WJetsHT70to100_2018': 380, 'WJetsHT100to200_2018': 381, 'WJetsHT200to400_2018': 382, 'WJetsHT400to600_2018': 383, 'WJetsHT600to800_2018': 384, 'WJetsHT800to1200_2018': 385, 'WJetsHT1200to2500_2018': 386, 'WJetsHT2500toInf_2018': 387, 'DYJetsToLL_2018': 388, 'DY1JetsToLL_2018': 389, 'DY2JetsToLL_2018': 390, 'DY3JetsToLL_2018': 391, 'DY4JetsToLL_2018': 392, 'SampleHTFake_2018': 393, 'VG_2018': 394, 'ZG_2018': 395, 'WG_2018': 396, 'TVX_2018': 397, 'TTGJets_2018': 398, 'TTZToQQ_2018': 399, 'TTZToLLNuNu_2018': 400, 'TTWJetsToQQ_2018': 401, 'TTWJetsToLNu_2018': 402, 'tZq_ll_4f_2018': 403, 'WrongSign_2018': 404, 'WWto2L2Nu_2018': 405, 'GluGluToWWToENuENu_2018': 406, 'GluGluToWWToMNuENu_2018': 407, 'GluGluToWWToTNuENu_2018': 408, 'STtW_top_2018': 409, 'GluGluHToWWTo2L2Nu_2018': 410, 'GluGluHToZZTo2L2Q_2018': 411, 'GluGluHToZZTo4L_2018': 412, 'GluGluHToTauTau_2018': 413, 'VBFHToWWTo2L2Nu_2018': 414, 'VBFHToTauTau_2018': 415, 'ttHToNonbb_2018': 416, 'VHToNonbb_2018': 417, 'OtherWS_2018': 418, 'Other_2018': 419, 'WWTo2L2Nu_DoubleScattering_2018': 420, 'WWW_4F_2018': 421, 'WWZTo3L1Nu2Q_2018': 422, 'WZZ_2018': 423, 'ZZZ_2018': 424, 'WWG_2018': 425, 'TTTo2L2Nu_2018': 426, 'WZ_2018': 427, 'FakeMu_2018': 428, 'DataMu_2018': 429, 'DataMuA_2018': 430, 'DataMuB_2018': 431, 'DataMuC_2018': 432, 'DataMuD_2018': 433, 'FakeEle_2018': 434, 'FakeMuPromptTau_2018': 435, 'PromptMuFakeTau_2018': 436, 'FakeMuFakeTau_2018': 437, 'DataEle_2018': 438, 'DataEleA_2018': 439, 'DataEleB_2018': 440, 'DataEleC_2018': 441, 'DataEleD_2018': 442, 'FakeElePromptTau_2018': 443, 'PromptEleFakeTau_2018': 444, 'FakeEleFakeTau_2018': 445, 'DataHT_2018': 446, 'DataHTA_2018': 447, 'DataHTB_2018': 448, 'DataHTC_2018': 449, 'DataHTD_2018': 450}

## Define samples to be processed

In [13]:


aggregated_samples = {
                      'Other': [WWTo2L2Nu_DoubleScattering_2017, WWW_4F_2017, WWZTo3L1Nu2Q_2017, WZZ_2017, ZZZ_2017, WWG_2017], 
                      'tVX': [TTGJets_2017, TTZToQQ_2017, TTZToLLNuNu_2017, TTWJetsToQQ_2017, TTWJetsToLNu_2017, tZq_ll_4f_2017], 
                      'QCD_ssWW': [WpWpJJ_QCD_2017],
                      'Vgamma':  [ZG_2017, WG_2017],
                      'ZZ': [ZZTo2L2Nu_2017, ZZJJTo4L_EWK_2017, ZZJJTo4L_QCD_2017, GluGluToContinToZZTo2e2nu_2017, GluGluToContinToZZTo2e2mu_2017, GluGluToContinToZZTo2e2tau_2017, GluGluToContinToZZTo2mu2nu_2017, GluGluToContinToZZTo2mu2tau_2017, GluGluToContinToZZTo4e_2017, GluGluToContinToZZTo4mu_2017, GluGluToContinToZZTo4tau_2017], 
                      'WZ': [WZ_2017], 
                      'VBS_ssWW': [VBS_SSWW_LL_SM_2017, VBS_SSWW_TL_SM_2017, VBS_SSWW_TT_SM_2017], 
                      #'VBS ssWW cW (BSM + INT)': [VBS_SSWW_cW_BSM_2017, VBS_SSWW_cW_INT_2017],
                      #'VBS ssWW cHW (BSM + INT)': [VBS_SSWW_cHW_BSM_2017, VBS_SSWW_cHW_INT_2017],
                      'DY_jets': [DY1JetsToLL_2017, DY2JetsToLL_2017, DY3JetsToLL_2017, DY4JetsToLL_2017, DYJetsToLLM5to50_2017],
                      'Wrong_Sign': [WWto2L2Nu_2017, GluGluToWWToENuENu_2017, GluGluToWWToENuMNu2017, 
                                     GluGluToWWToENuTNu2017, GluGluToWWToMNuENu_2017, GluGluToWWToMNuMNu2017, 
                                     GluGluToWWToMNuTNu2017, GluGluToWWToTNuENu_2017, GluGluToWWToTNuMNu2017, 
                                     GluGluToWWToTNuTNu2017, STtW_top_2017, 
                                        GluGluHToWWTo2L2Nu_2017, 
                                        #GluGluHToZZTo2L2Q_2017,
                                        GluGluHToZZTo4L_2017,
                                        GluGluHToTauTau_2017, 
                                        VBFHToWWTo2L2Nu_2017, VBFHToTauTau_2017, ttHToNonbb_2017, VHToNonbb_2017], 
                      'ttbar DiLep': [TTTo2L2Nu_2017],  
                      'Data': [DataEleB_2017, DataEleC_2017, DataEleD_2017, DataEleE_2017, DataEleF_2017, DataMuB_2017, DataMuC_2017, DataMuD_2017, DataMuE_2017, DataMuF_2017], 
                     }

In [14]:
with open('d.json', 'r') as fp:
    file_dict = json.load(fp)

In [15]:
chain = []
proxy = redirector
max_lines = maxNfilespersample

for v in aggregated_samples.values():
    for s in v:
        files = file_dict[s.label] 
        for i,line in enumerate(files):
            if i < max_lines:
                chain.append("root://" + proxy + "/" + line)
            i += 1
#chain

In [16]:
#chain

## Define the dataframe starting from a chain of root files

In [17]:
#!xrdcp root://xrootd-cms.infn.it//store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root .

In [18]:
#proxy = "root://212.189.205.221:30443//root://xrootd-cms.infn.it/"
proxy = "root://xrootd-cms.infn.it/"

chain = [proxy + "/store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]
#chain = ["5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"] #VBS_SSWW_LL_2017

if distributed == True:
    df = RDataFrame("Events", chain, npartitions=4, daskclient=client)
else:
    df = RDataFrame("Events", chain).Range(10)

## Define preselection flow - MC

In [19]:
%%time

df_year = df.DefinePerSample("Year","GetYear(rdfslot_, rdfsampleinfo_)")
df_sample = df_year.DefinePerSample("Sample", "GetSample(rdfslot_, rdfsampleinfo_)")

nevents_df = {}
gen_weight_df = {}
#q2_weight = {}
#ps_weight = {}
#pdf_weight = {}
df_samples = {}

#### MCWeight_writer #### 
for k,v in aggregated_samples.items():
    for sample in v:
        s = sample.label
        df_samples[s] = df_sample.Filter("Sample == {}".format(sampleDict[s]))
        nevents_df[s] = df_samples[s].Count()
        gen_weight_df[s] = df_samples[s].Sum("Generator_weight")
        #q2_weight[s] = [df_samples[s].Sum("LHEScaleWeight[{}]".format(j)) for j in range(0,7)]
        #ps_weight[s] = [df_samples[s].Sum("PSWeight[{}]".format(j)) for j in range(0,4)]
        #pdf_weight[s] = [df_samples[s].Sum("W_Pdf[{}]".format(j)) for j in range(0,15)]


#### MET HLT Filter ####   #### working only for 2017 and 2018 (2016 requires a column that is not present in 2017 nd 2018, i.e. HLT_IsoTkMu24
df_METHLTFilter = df_sample.Define("METHLTFilter", "MET_HLT_Filter(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_Ele27_WPTight_Gsf,  HLT_Ele32_WPTight_Gsf, HLT_IsoMu24, HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200)")\
                         .Filter("METHLTFilter == true", "MET HLT Filter")


#### preselection #####
df_preselection = df_METHLTFilter.Filter("PV_ndof> 4 && abs(PV_z) < 20 && hypot(PV_x, PV_y)<2", "Good vertex")\
                                 .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")


#### lepSF #####
df_LepSF = df_preselection.Define("ElectronSFs", "ElectronSFs(Electron_pt, Electron_eta, Electron_pdgId, Year)")\
                          .Define("Electron_effSF", "getFlattenedMatrixColumn(ElectronSFs, 3, 0)")\
                          .Define("Electron_effSF_errUp", "getFlattenedMatrixColumn(ElectronSFs, 3, 1)")\
                          .Define("Electron_effSF_errDown", "getFlattenedMatrixColumn(ElectronSFs, 3, 2)")\
                          .Define("MuonSFs", "MuonSFs(Muon_pt, Muon_eta, Muon_pdgId, Year)")\
                          .Define("Muon_effSF", "getFlattenedMatrixColumn(MuonSFs, 3, 0)")\
                          .Define("Muon_effSF_errUp", "getFlattenedMatrixColumn(MuonSFs, 3, 1)")\
                          .Define("Muon_effSF_errDown", "getFlattenedMatrixColumn(MuonSFs, 3, 2)")


#### mht ####
df_mht = df_LepSF.Define("MHT_pt_phi", "MHT_pt_phi(Electron_pt, Electron_eta, Electron_phi, Electron_mass, Electron_miniPFRelIso_all, Muon_pt, Muon_eta, Muon_phi, Muon_mass, Muon_miniPFRelIso_all, Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_muonIdx1, Jet_muonIdx2, Jet_electronIdx1, Jet_electronIdx2, nJet)")\
                 .Define("MHT_pt", "MHT_pt_phi[0]").Define("MHT_phi", "MHT_pt_phi[1]")\


#### puWeight #####
df_puWeight = df_mht.Define("puWeights", "puWeight(Year, Pileup_nTrueInt)")\
                    .Define("puWeight", "puWeights[0]")\
                    .Define("puWeightUp", "puWeights[1]")\
                    .Define("puWeightDown", "puWeights[2]")


#df_puWeight = df_mht


#### prefCorr ####
df_prefCorr = df_puWeight.Define("prefCorrs","PrefCorr(Photon_pt, Photon_eta, Photon_jetIdx, Photon_electronIdx, Electron_pt, Electron_eta, Electron_jetIdx, Electron_photonIdx, Jet_pt, Jet_eta)")\
                         .Define("PrefireWeight", "prefCorrs[0]")\
                         .Define("PrefireWeightUp", "prefCorrs[1]")\
                         .Define("PrefireWeightDown", "prefCorrs[2]")


#### ht ####
# https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/htProducerCpp.py
df_ht = df_prefCorr.Define("HT_pt", "htProducer(nJet, Jet_pt)")


#### btagSF ##### preselection part2
# https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/btv/btagSFProducer.py
df_btagSF = df_ht.Define("btagSFs",  'btagSF(Jet_pt, Jet_eta, Jet_hadronFlavour, Jet_btagDeepFlavB, Year, \"M\")')\
                 .Define("Jet_btagSF_deepjet_M", "getMatrixColumn(btagSFs, 0)")\
                 .Define("Jet_btagSF_deepjet_M_up", "getMatrixColumn(btagSFs, 1)")\
                 .Define("Jet_btagSF_deepjet_M_down", "getMatrixColumn(btagSFs, 2)")
#df_btagSF = df_ht


#### muonScaleRes ####
# https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/common/muonScaleResProducer.py
## TO DO insert in CVMFS necessary libraries (BOOST)
df_muonScaleRes = df_btagSF.Define("muonCorrectedPTs", "muonScaleRes(Muon_pt, Muon_eta, Muon_phi, Muon_charge, Muon_nTrackerLayers, Muon_genPartIdx, GenPart_pt, Year)")\
                           .Define("Muon_corrected_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 0)")\
                           .Define("Muon_correctedUp_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 1)")\
                           .Define("Muon_correctedDown_pt", "getFlattenedMatrixColumn(muonCorrectedPTs, 3, 2)")

#### metCorrector and fatJetCorrector #### ####
# https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/jetmetHelperRun2.py
    # https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/fatJetUncertainties.py
    # https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/jetmetUncertainties.py
        # https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/JetReCalibrator.py
        # https://github.com/anpicci/nanoAOD-tools/blob/VBS_PG/python/postprocessing/modules/jme/jetSmearer.py
df_jme = df_muonScaleRes

allCutsReport = df.Report()
allCutsReport.Print()

CPU times: user 17.3 s, sys: 362 ms, total: 17.7 s
Wall time: 20.8 s


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [24]:
#df_jme.Display(["Jet_pt", "Jet_btagSF_deepjet_M", "Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down"]).Print()
df_jme.Display(["Jet_eta",  "Jet_btagSF_deepjet_M", "Jet_btagSF_deepjet_M_up","Jet_btagSF_deepjet_M_down"]).Print()

Original eta: 2.42822
New eta: 2.399

sf is 1.2529
sf_up is 1.42595
sf_down is 1.07986
Original eta: -3.32324
New eta: -2.399

sf is 1.26626
sf_up is 1.43361
sf_down is 1.09892
Original eta: 1.36572
New eta: 1.36572

sf is 1.26725
sf_up is 1.41185
sf_down is 1.12264
Original eta: 1.52148
New eta: 1.52148

sf is 1.21626
sf_up is 1.34849
sf_down is 1.08402
Original eta: -0.81189
New eta: -0.81189

sf is 1.23244
sf_up is 1.41206
sf_down is 1.05283
Original eta: 2.56641
New eta: 2.399

sf is 1.25493
sf_up is 1.42721
sf_down is 1.08264
Original eta: 0.0454483
New eta: 0.0454483

sf is 1.24183
sf_up is 1.37911
sf_down is 1.10455
Original eta: 0.195038
New eta: 0.195038

sf is 1.23765
sf_up is 1.37402
sf_down is 1.10129
Original eta: -1.86743
New eta: -1.86743

sf is 1.25218
sf_up is 1.4255
sf_down is 1.07887
Original eta: -2.25879
New eta: -2.25879

sf is 1.26143
sf_up is 1.43105
sf_down is 1.09182
Original eta: -1.36694
New eta: -1.36694

sf is 1.26389
sf_up is 1.43239
sf_down is 1.09539
Or

In [23]:
%%capture
import ROOT
df_classic = ROOT.RDataFrame("Events", "https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root")
#df_classic.Display(["Jet_pt", "Jet_eta", "Jet_hadronFlavour", "Jet_btagDeepFlavB","Jet_btagSF_deepjet_M"], nRows = 5).Print()
df_classic.Display(["Jet_eta",  "Jet_btagSF_deepjet_M", "Jet_btagSF_deepjet_M_up","Jet_btagSF_deepjet_M_down"], nRows = 5).Print()

+-----+--------------+----------------------+-------------------------+---------------------------+
| Row | Jet_eta      | Jet_btagSF_deepjet_M | Jet_btagSF_deepjet_M_up | Jet_btagSF_deepjet_M_down | 
+-----+--------------+----------------------+-------------------------+---------------------------+
| 0   | 2.42822f     | 1.25290f             | 1.42595f                | 1.07986f                  | 
|     | -3.32324f    | 1.26626f             | 1.43361f                | 1.09892f                  | 
|     | 1.36572f     | 1.26725f             | 1.41185f                | 1.12264f                  | 
|     | 1.52148f     | 1.21626f             | 1.34849f                | 1.08402f                  | 
+-----+--------------+----------------------+-------------------------+---------------------------+
| 1   | -0.811890f   | 1.23244f             | 1.41206f                | 1.05283f                  | 
|     | 2.56641f     | 1.25493f             | 1.42721f                | 1.08264f              

## Define preselection flow - Data

In [43]:
for k,v in nevents_df.items():
    nevents_df[k] = v.GetValue()
for k,v in gen_weight_df.items():
    gen_weight_df[k] = v.GetValue()
with open('nEventsDict.json', 'w') as fp:
    json.dump(nevents_df, fp)
with open('genWeightDict.json', 'w') as fp:
    json.dump(gen_weight_df, fp)

In [ ]:
branches = [
    "nElectron","Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass", "Electron_charge", "Electron_pdgId", "Electron_jetRelIso", "Electron_mvaFall17V2Iso_WPL", "Electron_mvaFall17V2Iso_WP90", "Electron_effSF", "Electron_effSF_errUp", "Electron_effSF_errDown",
    "nMuon","Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge", "Muon_pdgId", "Muon_tightId", "Muon_looseId", "Muon_pfRelIso04_all", "Muon_effSF", "Muon_effSF_errUp", "Muon_effSF_errDown",
    "nTau","Tau_pt", "Tau_eta", "Tau_phi", "Tau_mass", "Tau_charge", "Tau_leadTkPtOverTauPt", "Tau_decayMode", "Tau_neutralIso", "Tau_idDeepTau2017v2p1VSjet", "Tau_idDeepTau2017v2p1VSe", "Tau_idDeepTau2017v2p1VSmu", "Tau_genPartFlav",
    "nJet","Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass", "Jet_jetId", "Jet_puId", "Jet_btagDeepFlavB", "Jet_btagDeepB", "Jet_btagCSVV2", "Jet_partonFlavour", "Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down", "Jet_btagSF_deepjet_M",
    "PrefireWeight", "PrefireWeightUp", "PrefireWeightDown", "puWeight", "puWeightUp", "puWeightDown"
]

In [13]:
branchlist = ROOT.vector('string')()
for branchName in ["nMuon","Muon_pt"]:
    branchlist.push_back(branchName)
df_jme.Snapshot("testTree", "testFile.root", branchlist)

<cppyy.gbl.ROOT.RDF.RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > object at 0x56439c319650>

In [ ]:
#df_jme.Snapshot("testTree", "testFile.root")

In [12]:
df_test =  RDataFrame("testTree", "testFile.root")
#df_test.Display(["ElectronSFs", "Electron_effSF", "MuonSFs", "Muon_effSF"], nRows = 20).Print()
df_test.Display(["Jet_pt", "Jet_eta", "Jet_hadronFlavour", "Jet_btagDeepFlavB"]).Print()

NameError: name 'RDataFrame' is not defined

In [ ]:
#nevents = nevents_df.GetValue()
#gen_weight = gen_weight_df.GetValue()

In [12]:
%%capture
import ROOT
df_classic = ROOT.RDataFrame("Events", "https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root")
#df_classic.Display(["Jet_pt", "Jet_eta", "Jet_hadronFlavour", "Jet_btagDeepFlavB","Jet_btagSF_deepjet_M"], nRows = 5).Print()
df_classic.Display(["Jet_btagSF_deepjet_M", "Jet_btagSF_deepjet_M_up", "Jet_btagSF_deepjet_M_down"], nRows = 5).Print()

+-----+----------------------+-------------------------+---------------------------+
| Row | Jet_btagSF_deepjet_M | Jet_btagSF_deepjet_M_up | Jet_btagSF_deepjet_M_down | 
+-----+----------------------+-------------------------+---------------------------+
| 0   | 1.25290f             | 1.42595f                | 1.07986f                  | 
|     | 1.26626f             | 1.43361f                | 1.09892f                  | 
|     | 1.26725f             | 1.41185f                | 1.12264f                  | 
|     | 1.21626f             | 1.34849f                | 1.08402f                  | 
+-----+----------------------+-------------------------+---------------------------+
| 1   | 1.23244f             | 1.41206f                | 1.05283f                  | 
|     | 1.25493f             | 1.42721f                | 1.08264f                  | 
|     | 1.24183f             | 1.37911f                | 1.10455f                  | 
|     | 1.23765f             | 1.37402f                | 

In [1]:
%%capture
import ROOT
from dask.distributed import Client

Error in <TExMap::Add>: key 139851959278768 is not unique
Error in <TExMap::Add>: key 139851959803600 is not unique


In [2]:
sampleDict = {'WpWpJJ_EWK_2017': 0, 'WpWpJJ_QCD_2017': 1, 'VBS_SSWW_SM_2017': 2, 'VBS_SSWW_LL_SM_2017': 3, 'VBS_SSWW_TL_SM_2017': 4, 'VBS_SSWW_TT_SM_2017': 5, 'VBS_SSWW_cHW_2017': 6, 'VBS_SSWW_cHW_BSM_2017': 7, 'VBS_SSWW_cHW_INT_2017': 8, 'VBS_SSWW_cW_2017': 9, 'VBS_SSWW_cW_BSM_2017': 10, 'VBS_SSWW_cW_INT_2017': 11, 'VBS_SSWW_cW_cHW_2017': 12, 'VBS_SSWW_DIM6_2017': 13, 'VBS_SSWW_DIM6_SM_2017': 14, 'VBS_SSWW_cHW_SM_2017': 15, 'VBS_SSWW_cW_SM_2017': 16, 'VBS_SSWW_aQGC_2017': 17, 'VBS_SSWW_FS0_25_BSM_2017': 18, 'VBS_SSWW_FS0_25_INT_2017': 19, 'VBS_SSWW_FS0_25_SM_2017': 20, 'VBS_SSWW_FS0_25_2017': 21, 'VBS_SSWW_FS0_5_BSM_2017': 22, 'VBS_SSWW_FS0_5_INT_2017': 23, 'VBS_SSWW_FS0_5_SM_2017': 24, 'VBS_SSWW_FS0_5_2017': 25, 'VBS_SSWW_FS0_0_2017': 26, 'VBS_SSWW_FS1_50_BSM_2017': 27, 'VBS_SSWW_FS1_50_INT_2017': 28, 'VBS_SSWW_FS1_50_SM_2017': 29, 'VBS_SSWW_FS1_50_2017': 30, 'VBS_SSWW_FS1_10_BSM_2017': 31, 'VBS_SSWW_FS1_10_INT_2017': 32, 'VBS_SSWW_FS1_10_SM_2017': 33, 'VBS_SSWW_FS1_10_2017': 34, 'VBS_SSWW_FS1_0_2017': 35, 'VBS_SSWW_FM0_25_BSM_2017': 36, 'VBS_SSWW_FM0_25_INT_2017': 37, 'VBS_SSWW_FM0_25_SM_2017': 38, 'VBS_SSWW_FM0_25_2017': 39, 'VBS_SSWW_FM0_5_BSM_2017': 40, 'VBS_SSWW_FM0_5_INT_2017': 41, 'VBS_SSWW_FM0_5_SM_2017': 42, 'VBS_SSWW_FM0_5_2017': 43, 'VBS_SSWW_FM0_0_2017': 44, 'VBS_SSWW_FM1_25_BSM_2017': 45, 'VBS_SSWW_FM1_25_INT_2017': 46, 'VBS_SSWW_FM1_25_SM_2017': 47, 'VBS_SSWW_FM1_25_2017': 48, 'VBS_SSWW_FM1_5_BSM_2017': 49, 'VBS_SSWW_FM1_5_INT_2017': 50, 'VBS_SSWW_FM1_5_SM_2017': 51, 'VBS_SSWW_FM1_5_2017': 52, 'VBS_SSWW_FM1_0_2017': 53, 'VBS_SSWW_FM6_25_BSM_2017': 54, 'VBS_SSWW_FM6_25_INT_2017': 55, 'VBS_SSWW_FM6_25_SM_2017': 56, 'VBS_SSWW_FM6_25_2017': 57, 'VBS_SSWW_FM6_5_BSM_2017': 58, 'VBS_SSWW_FM6_5_INT_2017': 59, 'VBS_SSWW_FM6_5_SM_2017': 60, 'VBS_SSWW_FM6_5_2017': 61, 'VBS_SSWW_FM6_0_2017': 62, 'VBS_SSWW_FM7_50_BSM_2017': 63, 'VBS_SSWW_FM7_50_INT_2017': 64, 'VBS_SSWW_FM7_50_SM_2017': 65, 'VBS_SSWW_FM7_50_2017': 66, 'VBS_SSWW_FM7_10_BSM_2017': 67, 'VBS_SSWW_FM7_10_INT_2017': 68, 'VBS_SSWW_FM7_10_SM_2017': 69, 'VBS_SSWW_FM7_10_2017': 70, 'VBS_SSWW_FM7_0_2017': 71, 'VBS_SSWW_FT0_2p5_BSM_2017': 72, 'VBS_SSWW_FT0_2p5_INT_2017': 73, 'VBS_SSWW_FT0_2p5_SM_2017': 74, 'VBS_SSWW_FT0_2p5_2017': 75, 'VBS_SSWW_FT0_0p5_BSM_2017': 76, 'VBS_SSWW_FT0_0p5_INT_2017': 77, 'VBS_SSWW_FT0_0p5_SM_2017': 78, 'VBS_SSWW_FT0_0p5_2017': 79, 'VBS_SSWW_FT0_0_2017': 80, 'VBS_SSWW_FT1_1_BSM_2017': 81, 'VBS_SSWW_FT1_1_INT_2017': 82, 'VBS_SSWW_FT1_1_SM_2017': 83, 'VBS_SSWW_FT1_1_2017': 84, 'VBS_SSWW_FT1_0p2_BSM_2017': 85, 'VBS_SSWW_FT1_0p2_INT_2017': 86, 'VBS_SSWW_FT1_0p2_SM_2017': 87, 'VBS_SSWW_FT1_0p2_2017': 88, 'VBS_SSWW_FT1_0_2017': 89, 'VBS_SSWW_FT2_2p5_BSM_2017': 90, 'VBS_SSWW_FT2_2p5_INT_2017': 91, 'VBS_SSWW_FT2_2p5_SM_2017': 92, 'VBS_SSWW_FT2_2p5_2017': 93, 'VBS_SSWW_FT2_0p5_BSM_2017': 94, 'VBS_SSWW_FT2_0p5_INT_2017': 95, 'VBS_SSWW_FT2_0p5_SM_2017': 96, 'VBS_SSWW_FT2_0p5_2017': 97, 'VBS_SSWW_FT2_0_2017': 98, 'DHiggsToWW_2017': 99, 'sm_2017': 100, 'sm_lin_quad_cW_2017': 101, 'quad_cW_2017': 102, 'sm_lin_quad_cHW_2017': 103, 'quad_cHW_2017': 104, 'QCD_2017': 105, 'QCDHT_100to200_2017': 106, 'QCDHT_200to300_2017': 107, 'QCDHT_300to500_2017': 108, 'QCDHT_500to700_2017': 109, 'QCDHT_700to1000_2017': 110, 'QCDHT_1000to1500_2017': 111, 'QCDHT_1500to2000_2017': 112, 'QCDHT_2000toInf_2017': 113, 'ZZtoLep_2017': 114, 'ZZTo2L2Nu_2017': 115, 'ZZJJTo4L_EWK_2017': 116, 'ZZJJTo4L_QCD_2017': 117, 'GluGluToContinToZZTo2e2nu_2017': 118, 'GluGluToContinToZZTo2e2mu_2017': 119, 'GluGluToContinToZZTo2e2tau_2017': 120, 'GluGluToContinToZZTo2mu2nu_2017': 121, 'GluGluToContinToZZTo2mu2tau_2017': 122, 'GluGluToContinToZZTo4e_2017': 123, 'GluGluToContinToZZTo4mu_2017': 124, 'GluGluToContinToZZTo4tau_2017': 125, 'GluGluToContinToZZTo4L_2017': 126, 'TT_2017': 127, 'TT_SemiLep2017': 128, 'TT_Had_2017': 129, 'TTTo2L2Nu_2017': 130, 'TT_beff_2017': 131, 'WJets_2017': 132, 'WJetsHT70to100_2017': 133, 'WJetsHT100to200_2017': 134, 'WJetsHT200to400_2017': 135, 'WJetsHT400to600_2017': 136, 'WJetsHT600to800_2017': 137, 'WJetsHT800to1200_2017': 138, 'WJetsHT1200to2500_2017': 139, 'WJetsHT2500toInf_2017': 140, 'WJets_Fake_2017': 141, 'WJetsHT70to100_Fake_2017': 142, 'WJetsHT100to200_Fake_2017': 143, 'WJetsHT200to400_Fake_2017': 144, 'WJetsHT400to600_Fake_2017': 145, 'WJetsHT600to800_Fake_2017': 146, 'WJetsHT800to1200_Fake_2017': 147, 'WJetsHT1200to2500_Fake_2017': 148, 'WJetsHT2500toInf_Fake_2017': 149, 'DYJetsToLL_2017': 150, 'DY1JetsToLL_2017': 151, 'DY2JetsToLL_2017': 152, 'DY3JetsToLL_2017': 153, 'DY4JetsToLL_2017': 154, 'DYJetsToLLM5to50_2017': 155, 'DYJetsToLL_Fake_2017': 156, 'DY1JetsToLL_Fake_2017': 157, 'DY2JetsToLL_Fake_2017': 158, 'DY3JetsToLL_Fake_2017': 159, 'DY4JetsToLL_Fake_2017': 160, 'VG_2017': 161, 'ZG_2017': 162, 'WG_2017': 163, 'TVX_2017': 164, 'TTGJets_2017': 165, 'TTZToQQ_2017': 166, 'TTZToLLNuNu_2017': 167, 'TTWJetsToQQ_2017': 168, 'TTWJetsToLNu_2017': 169, 'tZq_ll_4f_2017': 170, 'WrongSign_2017': 171, 'WWto2L2Nu_2017': 172, 'GluGluToWWToENuENu_2017': 173, 'GluGluToWWToENuMNu2017': 174, 'GluGluToWWToENuTNu2017': 175, 'GluGluToWWToMNuENu_2017': 176, 'GluGluToWWToMNuMNu2017': 177, 'GluGluToWWToMNuTNu2017': 178, 'GluGluToWWToTNuENu_2017': 179, 'GluGluToWWToTNuMNu2017': 180, 'GluGluToWWToTNuTNu2017': 181, 'STtW_top_2017': 182, 'GluGluHToWWTo2L2Nu_2017': 183, 'GluGluHToZZTo2L2Q_2017': 184, 'GluGluHToZZTo4L_2017': 185, 'GluGluHToTauTau_2017': 186, 'VBFHToWWTo2L2Nu_2017': 187, 'VBFHToTauTau_2017': 188, 'ttHToNonbb_2017': 189, 'VHToNonbb_2017': 190, 'OtherWS_2017': 191, 'Other_2017': 192, 'WWTo2L2Nu_DoubleScattering_2017': 193, 'WWW_4F_2017': 194, 'WWZTo3L1Nu2Q_2017': 195, 'WZZ_2017': 196, 'ZZZ_2017': 197, 'WWG_2017': 198, 'WZ_2017': 199, 'FakeMu_2017': 200, 'DataMu_2017': 201, 'DataMuB_2017': 202, 'DataMuC_2017': 203, 'DataMuD_2017': 204, 'DataMuE_2017': 205, 'DataMuF_2017': 206, 'DataMuFake_2017': 207, 'FakeEle_2017': 208, 'FakeMuPromptTau_2017': 209, 'PromptMuFakeTau_2017': 210, 'FakeMuFakeTau_2017': 211, 'DataEle_2017': 212, 'DataEleB_2017': 213, 'DataEleC_2017': 214, 'DataEleD_2017': 215, 'DataEleE_2017': 216, 'DataEleF_2017': 217, 'DataEleFake_2017': 218, 'MCFake_2017': 219, 'FakeElePromptTau_2017': 220, 'PromptEleFakeTau_2017': 221, 'FakeEleFakeTau_2017': 222, 'DataEleMu_2017': 223, 'FakeEleMu_2017': 224, 'DataTau_2017': 225, 'DataTauB_2017': 226, 'DataTauC_2017': 227, 'DataTauD_2017': 228, 'DataTauE_2017': 229, 'DataTauF_2017': 230, 'DataTauFake_2017': 231, 'DataHT_2017': 232, 'DataHTnoB_2017': 233, 'DataHTB_2017': 234, 'DataHTC_2017': 235, 'DataHTD_2017': 236, 'DataHTE_2017': 237, 'DataHTF_2017': 238, 'DataMET_2017': 239, 'DataMETB_2017': 240, 'DataMETC_2017': 241, 'DataMETD_2017': 242, 'DataMETE_2017': 243, 'DataMETF_2017': 244, 'SampleEleFake_2017': 245, 'SampleMuFake_2017': 246, 'SampleHTFake_2017': 247, 'SampleHTFakepart_2017': 248, 'WpWpJJ_EWK_2018': 249, 'WpWpJJ_QCD_2018': 250, 'VBS_SSWW_SM_2018': 251, 'VBS_SSWW_LL_SM_2018': 252, 'VBS_SSWW_TL_SM_2018': 253, 'VBS_SSWW_TT_SM_2018': 254, 'VBS_SSWW_cHW_2018': 255, 'VBS_SSWW_cHW_BSM_2018': 256, 'VBS_SSWW_cHW_INT_2018': 257, 'VBS_SSWW_cW_2018': 258, 'VBS_SSWW_cW_BSM_2018': 259, 'VBS_SSWW_cW_INT_2018': 260, 'VBS_SSWW_cW_cHW_2018': 261, 'VBS_SSWW_DIM6_2018': 262, 'VBS_SSWW_DIM6_SM_2018': 263, 'VBS_SSWW_cHW_SM_2018': 264, 'VBS_SSWW_cW_SM_2018': 265, 'VBS_SSWW_aQGC_2018': 266, 'VBS_SSWW_FS0_25_BSM_2018': 267, 'VBS_SSWW_FS0_25_INT_2018': 268, 'VBS_SSWW_FS0_25_SM_2018': 269, 'VBS_SSWW_FS0_25_2018': 270, 'VBS_SSWW_FS0_5_BSM_2018': 271, 'VBS_SSWW_FS0_5_INT_2018': 272, 'VBS_SSWW_FS0_5_SM_2018': 273, 'VBS_SSWW_FS0_5_2018': 274, 'VBS_SSWW_FS0_0_2018': 275, 'VBS_SSWW_FS1_50_BSM_2018': 276, 'VBS_SSWW_FS1_50_INT_2018': 277, 'VBS_SSWW_FS1_50_SM_2018': 278, 'VBS_SSWW_FS1_50_2018': 279, 'VBS_SSWW_FS1_10_BSM_2018': 280, 'VBS_SSWW_FS1_10_INT_2018': 281, 'VBS_SSWW_FS1_10_SM_2018': 282, 'VBS_SSWW_FS1_10_2018': 283, 'VBS_SSWW_FS1_0_2018': 284, 'VBS_SSWW_FM0_25_BSM_2018': 285, 'VBS_SSWW_FM0_25_INT_2018': 286, 'VBS_SSWW_FM0_25_SM_2018': 287, 'VBS_SSWW_FM0_25_2018': 288, 'VBS_SSWW_FM0_5_BSM_2018': 289, 'VBS_SSWW_FM0_5_INT_2018': 290, 'VBS_SSWW_FM0_5_SM_2018': 291, 'VBS_SSWW_FM0_5_2018': 292, 'VBS_SSWW_FM0_0_2018': 293, 'VBS_SSWW_FM1_25_BSM_2018': 294, 'VBS_SSWW_FM1_25_INT_2018': 295, 'VBS_SSWW_FM1_25_SM_2018': 296, 'VBS_SSWW_FM1_25_2018': 297, 'VBS_SSWW_FM1_5_BSM_2018': 298, 'VBS_SSWW_FM1_5_INT_2018': 299, 'VBS_SSWW_FM1_5_SM_2018': 300, 'VBS_SSWW_FM1_5_2018': 301, 'VBS_SSWW_FM1_0_2018': 302, 'VBS_SSWW_FM6_25_BSM_2018': 303, 'VBS_SSWW_FM6_25_INT_2018': 304, 'VBS_SSWW_FM6_25_SM_2018': 305, 'VBS_SSWW_FM6_25_2018': 306, 'VBS_SSWW_FM6_5_BSM_2018': 307, 'VBS_SSWW_FM6_5_INT_2018': 308, 'VBS_SSWW_FM6_5_SM_2018': 309, 'VBS_SSWW_FM6_5_2018': 310, 'VBS_SSWW_FM6_0_2018': 311, 'VBS_SSWW_FM7_50_BSM_2018': 312, 'VBS_SSWW_FM7_50_INT_2018': 313, 'VBS_SSWW_FM7_50_SM_2018': 314, 'VBS_SSWW_FM7_50_2018': 315, 'VBS_SSWW_FM7_10_BSM_2018': 316, 'VBS_SSWW_FM7_10_INT_2018': 317, 'VBS_SSWW_FM7_10_SM_2018': 318, 'VBS_SSWW_FM7_10_2018': 319, 'VBS_SSWW_FM7_0_2018': 320, 'VBS_SSWW_FT0_2p5_BSM_2018': 321, 'VBS_SSWW_FT0_2p5_INT_2018': 322, 'VBS_SSWW_FT0_2p5_SM_2018': 323, 'VBS_SSWW_FT0_2p5_2018': 324, 'VBS_SSWW_FT0_0p5_BSM_2018': 325, 'VBS_SSWW_FT0_0p5_INT_2018': 326, 'VBS_SSWW_FT0_0p5_SM_2018': 327, 'VBS_SSWW_FT0_0p5_2018': 328, 'VBS_SSWW_FT0_0_2018': 329, 'VBS_SSWW_FT1_1_BSM_2018': 330, 'VBS_SSWW_FT1_1_INT_2018': 331, 'VBS_SSWW_FT1_1_SM_2018': 332, 'VBS_SSWW_FT1_1_2018': 333, 'VBS_SSWW_FT1_0p2_BSM_2018': 334, 'VBS_SSWW_FT1_0p2_INT_2018': 335, 'VBS_SSWW_FT1_0p2_SM_2018': 336, 'VBS_SSWW_FT1_0p2_2018': 337, 'VBS_SSWW_FT1_0_2018': 338, 'VBS_SSWW_FT2_2p5_BSM_2018': 339, 'VBS_SSWW_FT2_2p5_INT_2018': 340, 'VBS_SSWW_FT2_2p5_SM_2018': 341, 'VBS_SSWW_FT2_2p5_2018': 342, 'VBS_SSWW_FT2_0p5_BSM_2018': 343, 'VBS_SSWW_FT2_0p5_INT_2018': 344, 'VBS_SSWW_FT2_0p5_SM_2018': 345, 'VBS_SSWW_FT2_0p5_2018': 346, 'VBS_SSWW_FT2_0_2018': 347, 'DHiggsToWW_2018': 348, 'sm_2018': 349, 'sm_lin_quad_cW_2018': 350, 'quad_cW_2018': 351, 'sm_lin_quad_cHW_2018': 352, 'quad_cHW_2018': 353, 'QCD_2018': 354, 'QCDHT_100to200_2018': 355, 'QCDHT_200to300_2018': 356, 'QCDHT_300to500_2018': 357, 'QCDHT_500to700_2018': 358, 'QCDHT_700to1000_2018': 359, 'QCDHT_1000to1500_2018': 360, 'QCDHT_1500to2000_2018': 361, 'QCDHT_2000toInf_2018': 362, 'ZZtoLep_2018': 363, 'ZZTo2L2Nu_2018': 364, 'ZZJJTo4L_EWK_2018': 365, 'ZZJJTo4L_QCD_2018': 366, 'GluGluToContinToZZTo2e2nu_2018': 367, 'GluGluToContinToZZTo2e2mu_2018': 368, 'GluGluToContinToZZTo2e2tau_2018': 369, 'GluGluToContinToZZTo2mu2nu_2018': 370, 'GluGluToContinToZZTo2mu2tau_2018': 371, 'GluGluToContinToZZTo4e_2018': 372, 'GluGluToContinToZZTo4mu_2018': 373, 'GluGluToContinToZZTo4tau_2018': 374, 'GluGluToContinToZZTo4L_2018': 375, 'TT_2018': 376, 'TT_SemiLep_2018': 377, 'TT_Had_2018': 378, 'WJets_2018': 379, 'WJetsHT70to100_2018': 380, 'WJetsHT100to200_2018': 381, 'WJetsHT200to400_2018': 382, 'WJetsHT400to600_2018': 383, 'WJetsHT600to800_2018': 384, 'WJetsHT800to1200_2018': 385, 'WJetsHT1200to2500_2018': 386, 'WJetsHT2500toInf_2018': 387, 'DYJetsToLL_2018': 388, 'DY1JetsToLL_2018': 389, 'DY2JetsToLL_2018': 390, 'DY3JetsToLL_2018': 391, 'DY4JetsToLL_2018': 392, 'SampleHTFake_2018': 393, 'VG_2018': 394, 'ZG_2018': 395, 'WG_2018': 396, 'TVX_2018': 397, 'TTGJets_2018': 398, 'TTZToQQ_2018': 399, 'TTZToLLNuNu_2018': 400, 'TTWJetsToQQ_2018': 401, 'TTWJetsToLNu_2018': 402, 'tZq_ll_4f_2018': 403, 'WrongSign_2018': 404, 'WWto2L2Nu_2018': 405, 'GluGluToWWToENuENu_2018': 406, 'GluGluToWWToMNuENu_2018': 407, 'GluGluToWWToTNuENu_2018': 408, 'STtW_top_2018': 409, 'GluGluHToWWTo2L2Nu_2018': 410, 'GluGluHToZZTo2L2Q_2018': 411, 'GluGluHToZZTo4L_2018': 412, 'GluGluHToTauTau_2018': 413, 'VBFHToWWTo2L2Nu_2018': 414, 'VBFHToTauTau_2018': 415, 'ttHToNonbb_2018': 416, 'VHToNonbb_2018': 417, 'OtherWS_2018': 418, 'Other_2018': 419, 'WWTo2L2Nu_DoubleScattering_2018': 420, 'WWW_4F_2018': 421, 'WWZTo3L1Nu2Q_2018': 422, 'WZZ_2018': 423, 'ZZZ_2018': 424, 'WWG_2018': 425, 'TTTo2L2Nu_2018': 426, 'WZ_2018': 427, 'FakeMu_2018': 428, 'DataMu_2018': 429, 'DataMuA_2018': 430, 'DataMuB_2018': 431, 'DataMuC_2018': 432, 'DataMuD_2018': 433, 'FakeEle_2018': 434, 'FakeMuPromptTau_2018': 435, 'PromptMuFakeTau_2018': 436, 'FakeMuFakeTau_2018': 437, 'DataEle_2018': 438, 'DataEleA_2018': 439, 'DataEleB_2018': 440, 'DataEleC_2018': 441, 'DataEleD_2018': 442, 'FakeElePromptTau_2018': 443, 'PromptEleFakeTau_2018': 444, 'FakeEleFakeTau_2018': 445, 'DataHT_2018': 446, 'DataHTA_2018': 447, 'DataHTB_2018': 448, 'DataHTC_2018': 449, 'DataHTD_2018': 450}

In [3]:
RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
client = Client(address="tcp://127.0.0.1:23708")

chain = ['https://ttedesch.web.cern.ch/ttedesch/testAF/VBS_SSWW_LL_SM_2017/tree_hadd_2.root', ##sample 3
         'https://ttedesch.web.cern.ch/ttedesch/testAF/ZZZ_2017/tree_hadd_1.root',  ###sample 197
        ]
df = ROOT.RDataFrame("Events", chain) 
df = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame("Events", chain, npartitions=10, daskclient=client)

nEvents = {}

df_dummy = df.Define("dummy", "1")

for k,v in sampleDict.items():
    if v == 3 or v==197:
        #nEvents[v] = df.Filter("Sample == {}".format(v)).Count()
        nEvents[v] = df_dummy.Filter("Sample == {}".format(v)).Sum("dummy")

/opt/conda/lib/python3.9/site-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.20.3 | 1.21.2    | 1.21.2  |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
%%time
nEvents[3].GetValue()

/cvmfs/cms.dodas.infn.it/miniconda3/envs/cms-dodas/lib/root/DistRDF/Backends/Base.py:138: UserWarning: Number of partitions is greater than number of clusters in the dataset. Using 4 partition(s)
  ranges = headnode.build_clustered_ranges()


CPU times: user 1.77 s, sys: 224 ms, total: 2 s
Wall time: 14.9 s


2205.0

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [5]:
%%time
nEvents[197].GetValue()

CPU times: user 101 µs, sys: 31 µs, total: 132 µs
Wall time: 144 µs


7262.0

In [7]:
%%time
nEvents[400].GetValue()

KeyError: 400

In [44]:
a

NameError: name 'a' is not defined